In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents = loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [5]:
embeddings=OllamaEmbeddings()
db=FAISS.from_documents(docs, embeddings)
db

In [6]:
## query
query = "How does the speaker describes the outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

### As a Retriver

In [9]:
retriver=db.as_retriever()
docs=retriver.invoke(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

### Similarity search with score
We are comparing Manhattan distance

In [10]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='18da8653-c47a-4f12-8a83-35942d166c65', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(14814.881)),
 (Document(id='e0725d5d-4a10-4caa-a556-0dc5601647cb', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we 

In [11]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.9328487515449524,
 -2.237520694732666,
 1.7996162176132202,
 2.0160648822784424,
 -0.7376236915588379,
 0.5833699703216553,
 1.6978814601898193,
 -0.45614802837371826,
 0.2290496528148651,
 -1.6061842441558838,
 2.305933713912964,
 -1.8664919137954712,
 -0.2522192597389221,
 1.1641789674758911,
 -1.0473759174346924,
 -1.3993191719055176,
 -0.26360708475112915,
 -0.25703561305999756,
 1.2118905782699585,
 -2.101625919342041,
 -0.4447159767150879,
 -0.6274381875991821,
 1.9736249446868896,
 -1.4206221103668213,
 1.9001141786575317,
 -1.1165330410003662,
 0.8969618082046509,
 -1.2937620878219604,
 0.03772372379899025,
 -0.7842666506767273,
 2.8778955936431885,
 -2.9497272968292236,
 -3.0455613136291504,
 4.200788974761963,
 1.9485772848129272,
 -5.077885627746582,
 -0.6478046178817749,
 1.8241877555847168,
 -0.6861655712127686,
 -0.5064525008201599,
 -0.46135544776916504,
 -3.2587454319000244,
 1.922196626663208,
 -0.0724671334028244,
 0.7118085026741028,
 0.20707359910011292,
 -0.4994

In [12]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='18da8653-c47a-4f12-8a83-35942d166c65', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='e0725d5d-4a10-4caa-a556-0dc5601647cb', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w

In [13]:
## Saving and loading
db.save_local("faiss_index")

In [16]:
new_db=FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)